<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df_inventory = pd.DataFrame({
    'Product': ['X1', 'X2', 'X3', 'X4', 'X5'],
    'StockLevel': [5, 8, 2, 15, 4],
    'ReorderLevel': [10, 6, 5, 10, 3]
})

In [ ]:
df_orders = pd.DataFrame({
    'OrderID': [101, 111, 121, 102, 132, 143, 104, 155, 166, 109],
    'Product': ['X1', 'X2', 'X1', 'X3', 'X4', 'X2', 'X1', 'X3', 'X5', 'X2'],
    'Quantity': [5, 10, 4, 6, 3, 7, 8, 9, 5, 6],
    'CustomerID': ['C1', 'C2', 'C3', 'C4', 'C5', 'C2', 'C1', 'C6', 'C3', 'C2']
})

In [ ]:
df_returns = pd.DataFrame({
    'OrderID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'Product': ['X1', 'X2', 'X1', 'X3', 'X4', 'X1', 'X2', 'X1', 'X3', 'X2'],
    'ReturnReason': ['Damaged', 'Expired', 'Damaged', 'Size Issue', 'Damaged',
                     'Expired', 'Damaged', 'Damaged', 'Expired', 'Expired']
})

In [ ]:
sus = df_inventory[df_inventory.StockLevel<df_inventory.ReorderLevel]
sus

,Product,StockLevel,ReorderLevel
0,X1,5,10
2,X3,2,5


In [ ]:
retSus = sus.merge(df_returns,on='Product',how='inner')
retSus

,Product,StockLevel,ReorderLevel,OrderID,ReturnReason
0,X1,5,10,101,Damaged
1,X1,5,10,103,Damaged
2,X1,5,10,106,Expired
3,X1,5,10,108,Damaged
4,X3,2,5,104,Size Issue
5,X3,2,5,109,Expired


In [ ]:
products  = list(retSus[((retSus.groupby('Product')['OrderID'].transform('count')>3))&(retSus.ReturnReason.isin(['Damaged','Expired']))]['Product'].drop_duplicates())

In [ ]:
df_orders['Flag'] = df_orders.Product.isin(products)

In [ ]:
df_orders

,OrderID,Product,Quantity,CustomerID,Flag
0,101,X1,5,C1,True
1,111,X2,10,C2,False
2,121,X1,4,C3,True
3,102,X3,6,C4,False
4,132,X4,3,C5,False
5,143,X2,7,C2,False
6,104,X1,8,C1,True
7,155,X3,9,C6,False
8,166,X5,5,C3,False
9,109,X2,6,C2,False


In [ ]:
x = df_orders[df_orders.Flag==True]
x['Tot_Quantity'] = x.groupby('CustomerID')['Quantity'].transform('sum')
def impactScore(score):
  if score>20:
    return 'High'
  elif score>10 and score<=20:
    return 'Medium'
  else:
    return 'Low'
x['ImpactScore'] = x['Tot_Quantity'].apply(impactScore)
x

/tmp/ipython-input-48-2867055840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Tot_Quantity'] = x.groupby('CustomerID')['Quantity'].transform('sum')
/tmp/ipython-input-48-2867055840.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ImpactScore'] = x['Tot_Quantity'].apply(impactScore)


,OrderID,Product,Quantity,CustomerID,Flag,Tot_Quantity,ImpactScore
0,101,X1,5,C1,True,13,Medium
2,121,X1,4,C3,True,4,Low
6,104,X1,8,C1,True,13,Medium


In [ ]:
reor = df_orders[df_orders.Flag==True].merge(df_returns,on=('OrderID','Product'),how='left',suffixes=('','_ret'))
reor.groupby(['Product','CustomerID']).agg({'Quantity':'sum','ReturnReason':(lambda x:(x.mode()[0] if not x.mode().empty else None))}).reset_index()

,Product,CustomerID,Quantity,ReturnReason
0,X1,C1,13,Damaged
1,X1,C3,4,None
